In [1]:
#%matplotlib inline
%matplotlib widget


In [2]:
import cv2 as cv2
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import trackpy as tp
from ipywidgets import HBox, Textarea, interact
import time
import imageio.v3 as iio

def l2n(x):
    return np.array(x)

In [3]:
def parseCircles(circles):
    x = []
    y = []
    r = []
    '''
    for i in range(len(circles)):
        x.append(circles[i,0,0])
        y.append(circles[i,0,1])
        r.append(circles[i,0,2])
    '''
    for i in circles[0, :]:#range(len(circles)):
        x.append(i[0])#x.append(circles[i,0,0])
        y.append(i[1])#y.append(circles[i,0,1])
        r.append(i[2])#r.append(circles[i,0,2])
    x,y,r = l2n(x), l2n(y),l2n(r)
    return x,y,r

In [4]:
def preProcessFrame(frame):
    #blur and grayout a frame
    
    # Convert to grayscale.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      
    # Convert to HSV and extract the saturation channel
    #sat = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)[:,:,0]

    # median filter
    #median = cv2.medianBlur(gray, 3)
    #gray_blurred = median
    # Blur using 3 * 3 kernel.
    blurSize = 10#5
    gray_blurred = cv2.blur(gray, (blurSize, blurSize))
    
    return gray_blurred

def detectCircles(frame,minRadius,maxRadius,param1=30,param2=25):
    
    minDist = int(0.9*maxRadius)#int(2.5*maxRadius)
    dp = 1
    # Apply Hough transform on the blurred image.
    circles = cv2.HoughCircles(frame, 
                cv2.HOUGH_GRADIENT,dp, minDist, param1 = param1,
                param2 = param2, minRadius = minRadius, maxRadius = maxRadius)

    return circles
    
def captureFrame(video,frame_id):
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
    ret, frame = video.read()
    if ret:
        return frame
    else:
        return ret

def drawCircles(circles,ax ):
    if circles is not None:  
    # Convert the circle parameters a, b and r to integers.
    #detected_circles = np.uint16(np.around(detected_circles))
    #fig, ax = plt.subplots() 
        for pt in circles[0, :]:
            a, b, r = pt[0], pt[1], pt[2]
            circle = plt.Circle((a, b), r, color='r',fill=False)
            ax.add_patch(circle)
            #ax.imshow(frame, cmap = 'gray')

def drawCirclesDf(df,ax ):
    
    
    for index, row in df.iterrows():
        x,y,r = row.x,row.y,row.r
        circle = plt.Circle((x, y), r, color='r',fill=False)
        ax.add_patch(circle)

def circlesToDataFrame(circles,frameNum):    
    x,y,r = parseCircles(circles)
    data = {'frame':[int(frameNum)]*len(circles[0]),'x':x,'y':y,'r':r}
    dft = pd.DataFrame(data = data)
    
    return dft


def cropImage(img,x0,y0,width,height):
    return img[y0:y0+height,x0:x0+width,:]
'''def cropImage(img,x0,y0,width,height):
    d = len(img.shape)
    if d==2: #grayscale
        return img[y0:y0+height,x0:x0+width]
    elif d==3: #rgb
        return img[y0:y0+height,x0:x0+width,:]
'''

'def cropImage(img,x0,y0,width,height):\n    d = len(img.shape)\n    if d==2: #grayscale\n        return img[y0:y0+height,x0:x0+width]\n    elif d==3: #rgb\n        return img[y0:y0+height,x0:x0+width,:]\n'

### Experimental settings

In [5]:
arenaSizePixels = 1080 # [pixels]
arenaSizeCm = 100 # [cm]

### Load experiments summary table

In [6]:
experiments = pd.read_csv('C:\\Users\\User\\Documents\\Eden\\locate_git_code\\two_circles_exp\\two_circles_exp5.csv',index_col=0)#,

## Locating params 

In [7]:
x0,y0 = 300,0
width0,height0 = 1150,1080

#frameNum = 1500
threshMin = 10
threshMax = 300

param1 = 31#100#25#35#30#25#25
param2 = 34#35

#frameSkip = 25


#### filter a sub-set of experiments

In [8]:
#expToTrack = experiments[(experiments["thirdpassiveDiameter[cm]"]==28)]
#expToTrack = experiments.iloc[range(36,50)]
expToTrack = experiments.iloc[range(75,83)]
#expToTrack = experiments[(experiments["date"]==20240207)]

In [9]:
expToTrack.head()
#expToTrack

,date,operator,bottype,stars,botNo,bots,firstpassiveDiameter[cm],secondpassiveDiameter[cm],thirdpassiveDiameter[cm],folder,...,comments,backedup,motorVal,production,linkingFileName,botLocatingFileName,botLinkingFileName,miniVideoFileNamePack,locatingFileNamePack,linkingFileNamePack
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
NaN,20240603,Eden,fronter,NaN,5.0,1,12.0,12,24.0,D:\Eden\new_exp_test_15_11_23\3_6_24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,20240603,Eden,fronter,NaN,5.0,1,12.0,12,24.0,D:\Eden\new_exp_test_15_11_23\3_6_24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,20240603,Eden,fronter,NaN,5.0,1,12.0,12,24.0,D:\Eden\new_exp_test_15_11_23\3_6_24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,20240603,Eden,fronter,NaN,5.0,1,12.0,12,24.0,D:\Eden\new_exp_test_15_11_23\3_6_24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,20240604,Eden,fronter,NaN,11.0,1,12.0,12,24.0,D:\Eden\new_exp_test_15_11_23\3_6_24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Batch locate - will take a while

In [10]:
locatingFileNameSuffix = '_locatedPillarFull20240605'

In [13]:
globalStartTime = time.time()

locatedFileNames = []

#diameter_pixel_thresh={7:[30,40],12:[58,72],18:[80,100],28:[136,151],32:[151,181]}
diameter_pixel_thresh={7:[30,40],10:[45,53],12:[55,65],14:[70,80],16:[76,85],18:[90,111],24:[121,140],28:[145,158],32:[151,181]}
xc,yc,wc,hc=425,0,1075,1080
first_time=True
multy_circ=False
for index,row in (expToTrack.iterrows()):
    first_time=True
    fileName =row.path
    print('Locating a circle in ' + fileName);
    video = cv2.VideoCapture(fileName)
    #video = iio.imiter(fileName)
    #frame_length=iio.improps(fileName).shape[0]
    #height=iio.improps(fileName).shape[1]
    #width=iio.improps(fileName).shape[2]
    #fps=25

    #get videos properties
    width  = int(video.get(cv2.CAP_PROP_FRAME_WIDTH ))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT ))
    fps = video.get(cv2.CAP_PROP_FPS)      # OpenCV v2.x used "CV_CAP_PROP_FPS"
    frameCount = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    pixelSize = arenaSizeCm/arenaSizePixels
    print(frameCount)
    #print(frame_length)
    
    #defined according the column from the experiments table
    #radiusMin = 32#int(row.minCircleRadius)
    #radiusMax = 40#int(row.maxCircleRadius)
    #first_diameter=int(row['firstpassiveDiameter[cm]'])
    #second_diameter=int(row['secondpassiveDiameter[cm]'])
    #third_diameter=int(row['thirdpassiveDiameter[cm]'])
    #diameters=[first_diameter,second_diameter,third_diameter]
    diameters=[12]
    print(diameters)
    #xc,yc = 1184,634 frameMin=6710 #
    frameMin = 0 #0#frameCount-100
    frameMax = frameCount#frameCount#frameMin+1000#frameCount#frameMin+500#300#frameCount#minFrame+3*60*24
    #frameRange = range(frameMin,frameMax,frameSkip)
    
    
    startTime = time.time()
    video.set(cv2.CAP_PROP_POS_FRAMES, frameMin)
    for diameter in diameters:
        print(diameter)
        radiusMin = diameter_pixel_thresh[diameter][0]#int(row.minCircleRadius)
        radiusMax = diameter_pixel_thresh[diameter][1]#int(row.maxCircleRadius
        data = {'frame':[],'x':[],'y':[],'r':[]}
        columns = ['frame','x','y','r']
        df = pd.DataFrame(data=data,columns=columns)
        df1=pd.DataFrame(data=data,columns=columns)
        df2=pd.DataFrame(data=data,columns=columns)
        ret = True
        for i in range(frameMin,100):
        #while ret:
            #video.grab()#captureFrame(video,frameNum)
            test=video.read()
            ret, img = video.read()
            if(ret): #if succesfffuly read an image
                frameNum = video.get(cv2.CAP_PROP_POS_FRAMES)
                #imgRed = img[:,:,0]

                #For large images, locate a cropped window

                imgCropped = cropImage(img,xc,yc,wc,hc)
                #imgCropped =cropImage(img,420,0,1080,1080)
                #Edge detection:
                #imgE = cv2.Canny(imgCropped,threshMin,threshMax)
                GrayImage = cv2.cvtColor(imgCropped, cv2.COLOR_BGR2GRAY)
                #Find circles:
                #circles = detectCircles(imgE,radiusMin, radiusMax,param1,param2)
                circles = detectCircles(GrayImage,radiusMin, radiusMax,param1,param2)
                #If circle found, add to pandas Dataframe:
                if circles is not None:        
                    dft = circlesToDataFrame(circles,frameNum)
                    #dft['x'] = dft['x']
                    #dft['y'] = dft['y']
                    if len(dft)>1:
                        multy_circ=True
                        if first_time==True:
                            df1=pd.concat([df1,dft.iloc[0].to_frame().T])#ignore_index=True)
                            df2=pd.concat([df2,dft.iloc[1].to_frame().T])#,ignore_index=True)
                            first_time=False
                        else:
                            if np.abs(df1['x'].iloc[0]-dft['x'].iloc[0])<10:
                                df1=pd.concat([df1,dft.iloc[0].to_frame().T])#,ignore_index=True)
                                df2=pd.concat([df2,dft.iloc[1].to_frame().T])#,ignore_index=True)
                            else:
                                df1=pd.concat([df1,dft.iloc[1].to_frame().T])#,ignore_index=True)
                                df2=pd.concat([df2,dft.iloc[0].to_frame().T])#,ignore_index=True)
                    else:
                        df = pd.concat([df,dft],ignore_index=True)
          #Save to CSV
            #Save the pandas locating dataframe a CSV file at the video's location
        if multy_circ==True:
            mean1=df1.mean(axis=0)
            df1=pd.concat([df1,mean1.to_frame().T],ignore_index=True)
            fileNameLoc1 = fileName+locatingFileNameSuffix+'_{}cm_10_frames_1.csv'.format(diameter)
            df1.to_csv(fileNameLoc1)
            mean2=df2.mean(axis=0)
            df2=pd.concat([df2,mean2.to_frame().T],ignore_index=True)
            fileNameLoc2 = fileName+locatingFileNameSuffix+'_{}cm_10_frames_2.csv'.format(diameter)
            df2.to_csv(fileNameLoc2)
        else:    
            mean=df.mean(axis=0)
            df=pd.concat([df,mean.to_frame().T],ignore_index=True)
            fileNameLoc = fileName+locatingFileNameSuffix+'_{}cm_10_frames.csv'.format(diameter)
            df.to_csv(fileNameLoc)        
            locatedFileNames.append(fileNameLoc)
    
    endTime = time.time()
    duration = endTime-startTime
    print('Locating ' + str(len(df)) + ' frames took ' + str(duration) + ' seconds.')
    
globalEndTime = time.time()
totalDuration = globalEndTime-globalStartTime
print('Locating ' + str(len(expToTrack)) + ' experiments took ' + str(totalDuration) + ' seconds.')

Locating a circle in D:\Eden\new_exp_test_15_11_23\3_6_24\C0853.MP4
17100
[12]
12
     frame       x       y          r
0      2.0  606.50  821.50  58.799999
1      4.0  607.50  821.50  58.900002
2      6.0  606.50  820.50  57.500000
3      8.0  608.50  820.50  58.900002
4     10.0  608.50  820.50  58.900002
..     ...     ...     ...        ...
96   194.0  606.50  821.50  59.400002
97   196.0  607.50  821.50  57.299999
98   198.0  606.50  820.50  58.900002
99   200.0  606.50  821.50  59.599998
100  101.0  607.23  821.39  58.639001

[101 rows x 4 columns]
     frame       x       y          r
0      2.0  266.50  665.50  57.700001
1      4.0  266.50  666.50  58.900002
2      6.0  265.50  666.50  59.700001
3      8.0  266.50  667.50  58.900002
4     10.0  266.50  665.50  57.700001
..     ...     ...     ...        ...
96   194.0  264.50  666.50  58.900002
97   196.0  265.50  668.50  58.900002
98   198.0  264.50  666.50  58.900002
99   200.0  264.50  668.50  57.799999
100  101.0  265.31  

In [14]:
dft.iloc[0].T

frame      2.000000
x        606.500000
y        817.500000
r         58.299999
Name: 0, dtype: float64